Setting up database 

In [1]:
import pandas as pd, csv, sqlite3

conn = sqlite3.connect('shark_info.db')
c = conn.cursor()
df = pd.read_csv("Mock_data.csv", low_memory=False)
df.to_sql(name='mockup_data', con=conn, if_exists='append', index=False)

conn.commit()

/Users/gabyodak/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/pandas/core/generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


Setting up machine learning dataframe

In [2]:
df.columns

Index(['index', 'Case Number', 'Date', 'Month', 'Day', 'Year', 'Type',
       'Country', 'Activity', 'Number of Victims ', 'Sex', 'Injury',
       'Fatal (Y/N)', 'Species'],
      dtype='object')

In [3]:
columns = [
    "Month", "Day", "Year", "Type", "Country", "Activity", 'Number of Victims ', 
    "Sex", "Injury", 'Fatal (Y/N)', "Species"
]
target = ['Fatal (Y/N)']


In [4]:
ml_df = df.loc[:, columns]
ml_df

,Month,Day,Year,Type,Country,Activity,Number of Victims,Sex,Injury,Fatal (Y/N),Species
0,2,5,2020,Unprovoked,USA,Stand-Up Paddle boarding,I,U,N,N,Tiger
1,1,30,2020,Provoked,BAHAMAS,Floating,I,F,Y,N,Unknown
2,1,17,2020,Unprovoked,AUSTRALIA,Surfing,I,M,Y,N,Unknown
3,1,16,2020,Unprovoked,NEW ZEALAND,Surfing,I,F,Y,N,Broad_Nose_Seven_Gill
4,1,13,2020,Unprovoked,USA,Surfing,I,M,Y,N,Unknown
5,1,5,2020,Unprovoked,AUSTRALIA,Scuba diving,I,M,Y,Y,Great_White


Encoding features and target

In [5]:
# Encoding feature column
df_binary_encoded = pd.get_dummies(ml_df, columns = [
    "Type",
    "Country",
    "Activity",
    "Number of Victims ",
    "Sex",
    "Injury",
    "Species"
])
#Encoding target column
# Not fatal (N): 0
# Fatal (Y): 1
x = {'N': 0}   
df_binary_encoded = df_binary_encoded.replace(x)
y = {'Y': 1}
df_binary_encoded = df_binary_encoded.replace(y)

pd.set_option('display.max_columns', None)
df_binary_encoded

,Month,Day,Year,Fatal (Y/N),Type_Provoked,Type_Unprovoked,Country_AUSTRALIA,Country_BAHAMAS,Country_NEW ZEALAND,Country_USA,Activity_Floating,Activity_Scuba diving,Activity_Stand-Up Paddle boarding,Activity_Surfing,Number of Victims _I,Sex_F,Sex_M,Sex_U,Injury_N,Injury_Y,Species_Broad_Nose_Seven_Gill,Species_Great_White,Species_Tiger,Species_Unknown
0,2,5,2020,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,1,0,0,0,1,0
1,1,30,2020,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,0,0,1
2,1,17,2020,0,0,1,1,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,1
3,1,16,2020,0,0,1,0,0,1,0,0,0,0,1,1,1,0,0,0,1,1,0,0,0
4,1,13,2020,0,0,1,0,0,0,1,0,0,0,1,1,0,1,0,0,1,0,0,0,1
5,1,5,2020,1,0,1,1,0,0,0,0,1,0,0,1,0,1,0,0,1,0,1,0,0


Split into Training and Testing data

In [6]:
from sklearn.model_selection import train_test_split
# Creating the features
X = df_binary_encoded.drop(columns='Fatal (Y/N)')

# Creating the target
y = df_binary_encoded['Fatal (Y/N)']

In [7]:
# Normal train test split 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

Easy Ensemble AdaBoost Classifier Model

In [8]:
from imblearn.ensemble import EasyEnsembleClassifier

eec_model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [9]:
# Fitting the model
eec_model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [10]:
# Making predictions using the testing data
predictions = eec_model.predict(X_test)

In [13]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predictions)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])
cm_df

,Predicted 0,Predicted 1
Actual 0,1,1
Actual 1,0,0
